## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Upernavik,GL,72.7868,-56.1549,15.19,95,100,4.65,overcast clouds
1,1,Shenjiamen,CN,29.9576,122.2980,54.59,58,0,9.53,clear sky
2,2,Port Alfred,ZA,-33.5906,26.8910,80.96,73,73,13.24,broken clouds
3,3,Sola,VU,-13.8833,167.5500,86.00,84,75,4.56,broken clouds
4,4,Codrington,AU,-38.2667,141.9667,59.11,67,37,26.96,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Puerto Ayora,EC,-0.7393,-90.3518,72.00,97,94,3.00,light rain
11,11,Huarmey,PE,-10.0681,-78.1522,70.32,84,100,4.99,overcast clouds
31,31,Port Elizabeth,ZA,-33.9180,25.5701,73.40,88,74,4.61,broken clouds
41,41,Kapaa,US,22.0752,-159.3190,73.40,88,20,4.61,few clouds
48,48,Sao Filipe,CV,14.8961,-24.4956,73.09,68,100,11.63,overcast clouds
62,62,Mataura,NZ,-46.1927,168.8643,71.10,64,25,3.65,scattered clouds
67,67,Simplicio Mendes,BR,-7.8539,-41.9103,72.23,86,99,3.00,overcast clouds
71,71,Ciudad Bolivar,VE,8.1222,-63.5497,73.85,86,91,9.98,overcast clouds
80,80,Hilo,US,19.7297,-155.0900,73.40,78,1,5.75,clear sky
82,82,Araure,VE,9.5601,-69.2148,72.84,72,100,3.42,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             65
City                   65
Country                64
Lat                    65
Lng                    65
Max Temp               65
Humidity               65
Cloudiness             65
Wind Speed             65
Current Description    65
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cleaned_preferred_cities_df = preferred_cities_df.dropna(axis = 0)
cleaned_preferred_cities_df.count()

Unnamed: 0             64
City                   64
Country                64
Lat                    64
Lng                    64
Max Temp               64
Humidity               64
Cloudiness             64
Wind Speed             64
Current Description    64
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cleaned_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,72.00,light rain,-0.7393,-90.3518,
11,Huarmey,PE,70.32,overcast clouds,-10.0681,-78.1522,
31,Port Elizabeth,ZA,73.40,broken clouds,-33.9180,25.5701,
41,Kapaa,US,73.40,few clouds,22.0752,-159.3190,
48,Sao Filipe,CV,73.09,overcast clouds,14.8961,-24.4956,
62,Mataura,NZ,71.10,scattered clouds,-46.1927,168.8643,
67,Simplicio Mendes,BR,72.23,overcast clouds,-7.8539,-41.9103,
71,Ciudad Bolivar,VE,73.85,overcast clouds,8.1222,-63.5497,
80,Hilo,US,73.40,clear sky,19.7297,-155.0900,
82,Araure,VE,72.84,overcast clouds,9.5601,-69.2148,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url  = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,72.00,light rain,-0.7393,-90.3518,Castillo Galapagos
11,Huarmey,PE,70.32,overcast clouds,-10.0681,-78.1522,Hotel Miramar
31,Port Elizabeth,ZA,73.40,broken clouds,-33.9180,25.5701,39 On Nile Guest House
41,Kapaa,US,73.40,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
48,Sao Filipe,CV,73.09,overcast clouds,14.8961,-24.4956,Tortuga B&B
62,Mataura,NZ,71.10,scattered clouds,-46.1927,168.8643,Ellie's Villa
67,Simplicio Mendes,BR,72.23,overcast clouds,-7.8539,-41.9103,Fidalgo
71,Ciudad Bolivar,VE,73.85,overcast clouds,8.1222,-63.5497,Hotel Edi
80,Hilo,US,73.40,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
82,Araure,VE,72.84,overcast clouds,9.5601,-69.2148,Hotel Las Majaguas


In [10]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 6 to 671
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 64 non-null     object 
 1   Country              64 non-null     object 
 2   Max Temp             64 non-null     float64
 3   Current Description  64 non-null     object 
 4   Lat                  64 non-null     float64
 5   Lng                  64 non-null     float64
 6   Hotel Name           64 non-null     object 
dtypes: float64(3), object(4)
memory usage: 6.5+ KB


In [11]:
# cleaned_hotels_df = hotel_df.loc['Hotel Name'].values()
# 7. Drop the rows where there is no Hotel Name.
string_to_NaN_df = hotel_df['Hotel Name'].replace('',np.nan, inplace=True)
string_to_NaN_df

In [12]:
cleaned_hotels_df = hotel_df.dropna(axis = 0)
cleaned_hotels_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Puerto Ayora,EC,72.00,light rain,-0.7393,-90.3518,Castillo Galapagos
11,Huarmey,PE,70.32,overcast clouds,-10.0681,-78.1522,Hotel Miramar
31,Port Elizabeth,ZA,73.40,broken clouds,-33.9180,25.5701,39 On Nile Guest House
41,Kapaa,US,73.40,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
48,Sao Filipe,CV,73.09,overcast clouds,14.8961,-24.4956,Tortuga B&B


In [13]:
cleaned_hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 6 to 671
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 59 non-null     object 
 1   Country              59 non-null     object 
 2   Max Temp             59 non-null     float64
 3   Current Description  59 non-null     object 
 4   Lat                  59 non-null     float64
 5   Lng                  59 non-null     float64
 6   Hotel Name           59 non-null     object 
dtypes: float64(3), object(4)
memory usage: 3.7+ KB


In [14]:
# 8a. Create the output File (CSV)
import os.path
output_data_file = os.path.join("../Vacation_Search", "WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
cleaned_hotels_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotels_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotels_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = cleaned_hotels_df[["Lat", "Lng"]]
max_temp = cleaned_hotels_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))